In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import GridSearchCV
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.datasets import cifar10
cifar = cifar10.load_data()


In [2]:
# Load CIFAR-10 dataset
X, y = cifar['data'], cifar['target']

TypeError: tuple indices must be integers or slices, not str

In [ ]:
# Preprocess data and labels
X = X / 255.0  # Normalize pixel values to [0, 1]
le = LabelEncoder()
y = le.fit_transform(y)

In [10]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X = (X_train,y_train), y = (y_val,X_val),test_size=0.2, random_state=42)

# Define the CNN model
def create_model(num_filters=32, kernel_size=3, pool_size=2, num_hidden_units=128, learning_rate=0.001):
    model = Sequential()
    model.add(Conv2D(num_filters, kernel_size=(kernel_size, kernel_size), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
    model.add(Flatten())
    model.add(Dense(num_hidden_units, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model


TypeError: train_test_split() got an unexpected keyword argument 'X'

In [ ]:
# Define hyperparameter grid
param_grid = {
    'num_filters': [16, 32],
    'kernel_size': [3, 5],
    'pool_size': [2, 3],
    'num_hidden_units': [64, 128],
    'learning_rate': [0.001, 0.01]
}



In [ ]:
# Create KerasClassifier
model = keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=10, batch_size=32)

# Create GridSearchCV instance
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=2, n_jobs=-1)

In [ ]:
# Perform the grid search
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [ ]:
# Evaluate the best model on the validation set
y_pred = best_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy:.2f}')

In [1]:
# Use scikit-learn to grid search the weight initialization
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(init_mode='uniform'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_shape=(8,), kernel_initializer=init_mode, activation='relu'))
	model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)
# load dataset
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(model__init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

ModuleNotFoundError: No module named 'scikeras'